In [ ]:
import numpy as np
import pandas as pd
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')
movies.head()
credits.head()
movies = movies.merge(credits,on='title')
movies.head()
print(movies.columns)
movies = movies['original_language'].value_counts()
movies.info()
import pandas as pd

# Load the datasets
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

# Merge the datasets on 'title'
movies = movies.merge(credits, on='title')

# Now extract the required columns
movies = movies[['movie_id', 'cast', 'crew', 'title', 'overview', 'genres', 'keywords']]

# Check the output
print(movies.head())
movies.isnull().sum()
movies.dropna(inplace = True)
movies.duplicated().sum()
movies.iloc[0].genres
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        return L
movies['genres'] = movies['genres'].apply(convert)
movies.head()    
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()
movies['cast'][0]
from typing import Counter
def convert3(obj):
    L=[]
    Counter = 0
    for i in ast.literal_eval(obj):
      if Counter != 3:
        L.append(i['name'])
        Counter+=1
      else:
        break
    return L
movies['cast'] = movies['cast'].apply(convert3)
movies.head()
movies['crew'][0]
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
      if i['job'] == 'Director':
        L.append(i['name'])
      else:
        break
    return L
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()
movies['overview'][0]
movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)
movies.head()
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies.head()
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new_df = movies[['movie_id','title','tags']]
new_df
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
new_df.head()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()
vector[0]
cv.get_feature_names_out()
import nltk
!pip install nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
      y.append(ps.stem(i))
    return " ".join(y)
new_df['tags'] = new_df['tags'].apply(stem)
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity[0]
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
